In [1]:
from collections.abc import Iterator
from random import randint, choice, random

N = 8  # Board size


def heuristic(board: str) -> int:
    """The heuristic cost function h is the number of pairs of queens that are attacking each other;
    this will be zero only for solutions. (It counts as an attack if two pieces are in the same line,
    even if there is an intervening piece between them.)
    """
    h = 0

    for i in range(N):
        # Value in column i
        row_i = int(board[i])

        for j in range(i + 1, N):
            # Value in column j
            row_j = int(board[j])

            # Same row
            if row_i == row_j:
                h += 1

            # Same diagonal
            if (j - i) == abs(row_j - row_i):
                h += 1

    return h


def random_state() -> str:
    """Generate a random board state by placing one queen in each column at a random row position."""
    res = ''

    for col in range(N):
        res += str(randint(1, N))

    return res


def successors(current: str) -> list[str]:
    """Generate all the successors of a state, i.e. all possible states generated by moving a single queen
    to another square in the same column (so each state has 8 x 7 = 56 successors)."""
    successors = []

    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                new_board = current[:col] + str(new_row) + current[col + 1 :]
                successors.append(new_board)

    return successors


def successors_generator(current: str) -> Iterator[str]:
    """Generate successor states lazily using a generator.

    Memory-efficient alternative to successors() that yields states one at a time
    instead of creating the entire list in memory.
    """
    for col in range(N):
        current_row = int(current[col])

        for new_row in range(1, N + 1):
            if new_row != current_row:
                # Create new board with queen moved
                new_board = current[:col] + str(new_row) + current[col + 1 :]
                yield new_board


def random_successor(current: str) -> str:
    """Generate a random successor of the current state."""
    col = randint(0, N - 1)
    current_row = int(current[col])

    new_row = randint(1, N)
    # If the value is the same as the current one, try again
    while new_row == current_row:
        new_row = randint(1, N)

    return current[:col] + str(new_row) + current[col + 1 :]


def display(board: str) -> None:
    """Display the board state in a visual format."""
    grid = [['_'] * N for _ in range(N)]

    for col in range(N):
        current_row = int(board[col])
        grid[N - current_row][col] = 'X'

    for row in grid:
        print(' '.join(row))

In [74]:
from math import exp


def hill_climbing(initial_state: str) -> tuple[str, dict[str, int | bool]]:
    """Steepest-ascent hill climbing algorithm."""
    current, h_current = initial_state, heuristic(initial_state)
    steps = 0

    while True:
        children = [
            (successor, heuristic(successor)) for successor in successors(current)
        ]
        neighbor, h_neighbor = min(children, key=lambda x: x[1])
        steps += 1
        if h_neighbor >= h_current:
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats
        else:
            current, h_current = neighbor, h_neighbor


def hill_climbing_sideways_move(
    initial_state: str,
) -> tuple[str, dict[str, int | bool]]:
    """Hill climbing with sideways moves to escape local shoulders."""
    current, h_current = initial_state, heuristic(initial_state)
    sideways_move = 0
    steps = 0

    while True:
        children = [
            (successor, heuristic(successor)) for successor in successors(current)
        ]
        # Filter on the best heuristic value
        h_min = min(children, key=lambda x: x[1])[1]
        children = [child for child in children if child[1] == h_min]
        # We choose a random neighbor among the best ones
        neighbor, h_neighbor = choice(children)
        steps += 1
        if (h_neighbor > h_current) or (sideways_move == 100):
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats
        elif h_neighbor == h_current:
            sideways_move += 1
            current, h_current = neighbor, h_neighbor
        else:
            current, h_current = neighbor, h_neighbor


def first_choice_hill_climbing(initial_state: str) -> tuple[str, dict[str, int | bool]]:
    """First-choice hill climbing algorithm.

    Generates successors and accepts the first one that improves the current state.
    Useful when there are many successors."""
    current, h_current = initial_state, heuristic(initial_state)
    steps = 0

    while True:
        for child in successors_generator(current):
            h_child = heuristic(child)

            if h_child < h_current:
                steps += 1
                current, h_current = child, h_child
                break
        # Finally found the use of an 'else' on a 'for' loop, fuck yeah!
        # If we have generated all successors without finding a better one
        else:
            stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
            return current, stats


def random_restart_hill_climbing() -> tuple[str, dict[str, int | bool]]:
    """Hill climbing with random restarts.

    Repeatedly runs hill climbing from random initial states until a solution
    is found. Guarantees finding a solution eventually (with probability 1)."""
    restarts, total_steps = 0, 0

    while True:
        current_state = random_state()
        res, stats = hill_climbing(current_state)
        total_steps += stats['steps']
        if stats['h_final'] == 0:
            stats['restarts'], stats['total_steps'] = restarts, total_steps
            return res, stats
        restarts += 1


def random_restart_hill_climbing_sideways_move() -> tuple[str, dict[str, int | bool]]:
    """Hill climbing with sideways moves and random restarts."""
    restarts, total_steps = 0, 0

    while True:
        current_state = random_state()
        res, stats = hill_climbing_sideways_move(current_state)
        total_steps += stats['steps']
        if stats['h_final'] == 0:
            stats['restarts'], stats['total_steps'] = restarts, total_steps
            return res, stats
        restarts += 1


def simulated_annealing(
    initial_state: str,
    max_iterations: int = 1000,
    initial_temp: int = 100,
    alpha: float = 0.90,
) -> tuple[str, dict[str, int | bool]]:
    """Simulated annealing with exponential cooling."""
    current, h_current = initial_state, heuristic(initial_state)
    temperature = initial_temp
    steps = 0

    for _ in range(max_iterations):
        if h_current == 0:
            break

        child = random_successor(current)
        h_child = heuristic(child)
        delta_e = h_current - h_child

        # If the move improves the situation (i.e delta_e > 0), it is always accepted
        # Otherwise if the move is worse (delta_e < 0),
        # the algorithm accepts the move with some probability less than 1
        if (delta_e > 0) or (exp(delta_e / temperature) > random()):
            steps += 1
            current, h_current = child, h_child

        temperature *= alpha

    stats = {'steps': steps, 'h_final': h_current, 'success': h_current == 0}
    return current, stats


def local_beam_search(
    k_size: int, max_steps: int = 40
) -> tuple[str, dict[str, int | bool]]:
    """Local beam search algorithm

    Keeps track of k states rather than just one. It begins with k randomly generated states.
    At each step, all the successors of all k states are generated.
    If any one is a goal, the algorithm halts.
    Otherwise, it selects the k best successors from the complete list and repeats."""
    currents = [random_state() for _ in range(k_size)]
    best_h = 1000
    steps = 0

    # Make sure we don't loop forever
    while steps < max_steps:
        # Generate all successors
        children = [
            (successor, heuristic(successor))
            for current in currents
            for successor in successors(current)
        ]

        # Sort by heuristic
        children.sort(key=lambda x: x[1])
        current_best_state, current_best_h = children[0]
        steps += 1

        if current_best_h == 0:
            # State is goal state
            stats = {'steps': steps, 'h_final': 0, 'success': True}
            return current_best_state, stats

        if current_best_h < best_h:
            best_h = current_best_h
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Early stopping if no improvement
        if no_improvement_count >= 10:
            break

        # We keep the k best ones
        currents = [child[0] for child in children[:k_size]]

    stats = {'steps': steps, 'h_final': heuristic(currents[0]), 'success': False}
    return currents[0], stats

In [3]:
s = '43254323'
display(s)
print(s)

_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ X _ _ _ _
X _ _ _ X _ _ _
_ X _ _ _ X _ X
_ _ X _ _ _ X _
_ _ _ _ _ _ _ _
43254323


In [4]:
print('>>> Hill Climbing:')
res, stats = hill_climbing(s)
display(res)
print(res)
stats

>>> Hill Climbing:
_ _ _ _ _ _ _ _
_ _ _ _ _ _ X _
_ X _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ X
_ _ X _ _ _ _ _
_ _ _ _ X _ _ _
36251473


{'steps': 6, 'h_final': 1, 'success': False}

In [5]:
print('>>> Hill Climbing Sideways Move:')
res, stats = hill_climbing_sideways_move(s)
display(res)
print(res)
stats

>>> Hill Climbing Sideways Move:
_ X _ _ _ _ _ _
_ _ _ _ X _ _ _
_ _ _ _ _ _ X _
_ _ _ X _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ X _ _
_ _ X _ _ _ _ _
48157263


{'steps': 13, 'h_final': 0, 'success': True}

In [6]:
print('>>> First Choice Hill Climbing:')
res, stats = first_choice_hill_climbing(s)
display(res)
print(res)
stats

>>> First Choice Hill Climbing:
_ X _ _ _ _ _ _
_ _ _ _ _ X _ _
X _ _ _ _ _ _ _
_ _ X _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
68541723


{'steps': 11, 'h_final': 2, 'success': False}

In [7]:
print('>>> Simulated Annealing:')
res, stats = simulated_annealing(s)
display(res)
print(res)
stats

>>> Simulated Annealing:
_ _ _ _ _ X _ _
_ _ _ X _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ X _ _ _
_ _ _ _ _ _ _ X
_ X _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ X _ _ _ _ _
63175824


{'steps': 67, 'h_final': 0, 'success': True}

### From a random state :

In [8]:
r = random_state()
display(r)
print(r)

_ _ _ _ _ X _ X
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ _ _ _
_ X _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ X _ X _ _ _
_ _ _ _ _ _ X _
74262818


In [9]:
print('>>> Hill Climbing:')
res, stats = hill_climbing(r)
display(res)
print(res)
stats

>>> Hill Climbing:
_ _ _ _ _ _ _ X
X _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ X _ _ _ _
_ X _ _ _ X _ _
_ _ _ _ _ _ _ _
_ _ X _ X _ _ _
_ _ _ _ _ _ X _
74252418


{'steps': 3, 'h_final': 3, 'success': False}

In [10]:
print('>>> Hill Climbing Sideways Move:')
res, stats = hill_climbing_sideways_move(r)
display(res)
print(res)
stats

>>> Hill Climbing Sideways Move:
_ _ X _ _ _ _ _
_ _ _ _ _ X _ _
_ _ _ _ _ _ _ X
_ X _ _ _ _ _ _
_ _ _ X _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
35841726


{'steps': 32, 'h_final': 0, 'success': True}

In [11]:
print('>>> First choice Hill Climbing:')
res, stats = first_choice_hill_climbing(r)
display(res)
print(res)
stats

>>> First choice Hill Climbing:
_ _ _ _ _ _ _ X
_ X _ _ _ _ _ _
_ _ _ X _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ X _ _
_ _ _ _ _ _ _ _
_ _ X _ X _ _ _
_ _ _ _ _ _ X _
57262418


{'steps': 4, 'h_final': 2, 'success': False}

In [12]:
print('>>> Simulated Annealing:')
res, stats = simulated_annealing(r)
display(res)
print(res)
stats

>>> Simulated Annealing:
_ _ _ _ _ _ X _
_ X _ X _ _ _ _
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ _ _ X _ _
X _ _ _ _ _ _ _
_ _ X _ _ _ _ _
_ _ _ _ X _ _ _
37271485


{'steps': 122, 'h_final': 1, 'success': False}

In [13]:
# %timeit hill_climbing(r)
# %timeit hill_climbing_sideways_move(r)
# %timeit first_choice_hill_climbing(r)
# %timeit simulated_annealing(r)

### Random restart algorithms :

In [14]:
print('>>> Random restart Hill Climbing:')
res, stats = random_restart_hill_climbing()
display(res)
print(res)
stats

>>> Random restart Hill Climbing:
_ _ _ _ X _ _ _
_ _ X _ _ _ _ _
X _ _ _ _ _ _ _
_ _ _ _ _ X _ _
_ _ _ _ _ _ _ X
_ X _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ _ X _
63728514


{'steps': 6, 'h_final': 0, 'success': True, 'restarts': 11, 'total_steps': 51}

In [15]:
print('>>> Random restart Hill Climbing Sideways Move:')
res, stats = random_restart_hill_climbing_sideways_move()
display(res)
print(res)
stats

>>> Random restart Hill Climbing Sideways Move:
_ X _ _ _ _ _ _
_ _ _ _ _ _ X _
_ _ _ _ X _ _ _
_ _ _ _ _ _ _ X
X _ _ _ _ _ _ _
_ _ _ X _ _ _ _
_ _ _ _ _ X _ _
_ _ X _ _ _ _ _
48136275


{'steps': 12, 'h_final': 0, 'success': True, 'restarts': 1, 'total_steps': 21}

### Local Beam Search

In [77]:
print('>>> Local Beam Search:')
res, stats = local_beam_search(k_size=20)
display(res)
print(res)
stats

>>> Local Beam Search:
_ _ X _ _ _ _ _
_ _ _ _ _ _ _ X
_ _ _ X _ _ _ _
_ _ _ _ _ _ X _
X _ _ _ _ _ _ _
_ _ _ _ _ X _ _
_ X _ _ _ _ _ _
_ _ _ _ X _ _ _
42861357


{'steps': 3, 'h_final': 0, 'success': True}

In [76]:
sum([local_beam_search(k_size=20)[1]['success'] for _ in range(100)])

99

In [ ]:
# %timeit random_restart_hill_climbing()
# %timeit random_restart_hill_climbing_sideways_move()
# %timeit local_beam_search(k_size=20)

14.1 ms ± 1.77 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
12 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
47 ms ± 6.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing())
# len(res)
# # res

In [19]:
# res = set()
# while len(res) < 92:
#     res.add(random_restart_hill_climbing_sideways_move())
# len(res)
# # res